# 📌 **Protección contra ataques (CSRF, XSS, SQL Injection)**  

El objetivo de este tema es que aprendas cómo **proteger una aplicación Django** contra ataques comunes en aplicaciones web:  

✅ **CSRF (Cross-Site Request Forgery)** → Evita que un atacante haga peticiones en nombre de un usuario sin su consentimiento.  
✅ **XSS (Cross-Site Scripting)** → Previene la ejecución de scripts maliciosos en el navegador.  
✅ **SQL Injection** → Evita que usuarios malintencionados manipulen la base de datos a través de consultas SQL.  

---

## 🔹 **1. Protección contra CSRF (Cross-Site Request Forgery)**  

### 📍 **¿Qué es CSRF?**  
Un ataque CSRF ocurre cuando un usuario autenticado en un sitio web realiza una acción involuntaria porque un atacante **envió una petición maliciosa en su nombre**.  

💡 **Ejemplo:**  
1️⃣ Un usuario está autenticado en **tu sitio web** (por ejemplo, en una cuenta bancaria).  
2️⃣ Un atacante le envía un enlace malicioso que, si el usuario lo abre, hace que su navegador **envíe una petición POST** a la API para transferir dinero **sin su consentimiento**.  

### 📍 **Cómo Django previene CSRF**  

Django ya tiene protección **CSRF habilitada por defecto** en vistas basadas en plantillas HTML.  

📌 **Ejemplo en una plantilla Django (`form.html`):**  

```html
<form method="POST">
    {% csrf_token %}  <!-- Protege el formulario contra CSRF -->
    <input type="text" name="nombre" placeholder="Tu nombre">
    <button type="submit">Enviar</button>
</form>
```

**El token CSRF es obligatorio en formularios POST.**

### 📍 **Cómo proteger APIs con CSRF**  

Las APIs basadas en Django REST Framework (DRF) **no incluyen protección CSRF por defecto**. Si queremos habilitarla, usamos el permiso `CsrfExemptMixin` o `@csrf_exempt` para deshabilitarlo en ciertas vistas.  

📌 **Ejemplo en `views.py`:**  

```python
from django.views.decorators.csrf import csrf_exempt
from django.http import JsonResponse

@csrf_exempt
def mi_api_view(request):
    if request.method == "POST":
        return JsonResponse({"mensaje": "Solicitud POST recibida."})
    return JsonResponse({"mensaje": "Método no permitido."}, status=405)
```

📌 **Si usamos autenticación JWT en una API, podemos desactivar CSRF:**  

```python
REST_FRAMEWORK = {
    'DEFAULT_AUTHENTICATION_CLASSES': (
        'rest_framework_simplejwt.authentication.JWTAuthentication',
    ),
    'DEFAULT_PERMISSION_CLASSES': (
        'rest_framework.permissions.IsAuthenticated',
    ),
}
```

---

## 🔹 **2. Protección contra XSS (Cross-Site Scripting)**  

### 📍 **¿Qué es XSS?**  
Un ataque XSS ocurre cuando un atacante **inyecta código JavaScript malicioso** en una aplicación web.  

💡 **Ejemplo:**  
1️⃣ Un usuario **escribe un comentario en tu blog** con este código:  
   ```html
   <script>alert('Hackeado!');</script>
   ```  
2️⃣ Si la página no filtra este código, **se ejecutará en el navegador de otros usuarios**.  

### 📍 **Cómo Django previene XSS**  

Django **escapa** automáticamente las variables en las plantillas para evitar XSS.  

📌 **Ejemplo seguro en Django:**  

```html
<p>Hola, {{ usuario.nombre }}</p>
```

📌 **Si `usuario.nombre` contiene `<script>alert("Hackeado!")</script>`, Django lo escapa y lo muestra como texto:**  

```html
<p>Hola, &lt;script&gt;alert("Hackeado!")&lt;/script&gt;</p>
```

📌 **Si necesitas mostrar HTML seguro, usa `|safe`** (pero úsalo con cuidado):  

```html
<p>{{ contenido|safe }}</p>
```

📍 **Protección extra contra XSS en Django:**  

✔️ **Filtrar datos en formularios:** Usa `forms.CharField(strip_tags=True)` en formularios.  
✔️ **Evitar `safe` cuando sea posible:** No permitas HTML sin filtrar en entradas de usuarios.  

---

## 🔹 **3. Protección contra SQL Injection**  

### 📍 **¿Qué es SQL Injection?**  
Un ataque de **SQL Injection** ocurre cuando un usuario **modifica una consulta SQL** a través de la entrada de datos.  

💡 **Ejemplo de código vulnerable:**  

```python
def buscar_usuario(request):
    nombre = request.GET.get("nombre")
    consulta = f"SELECT * FROM usuarios WHERE nombre = '{nombre}'"
    resultado = ejecutar_sql(consulta)  # 🚨 ¡Peligroso!
```

📌 **Si un usuario malicioso ingresa:**  
```sql
' OR '1'='1
```
La consulta se convierte en:  
```sql
SELECT * FROM usuarios WHERE nombre = '' OR '1'='1'
```
Esto hace que **se devuelvan todos los usuarios** de la base de datos.  

### 📍 **Cómo Django previene SQL Injection**  

✔️ **Usar consultas ORM en lugar de SQL crudo:**  

```python
from .models import Usuario

def buscar_usuario(request):
    nombre = request.GET.get("nombre")
    usuarios = Usuario.objects.filter(nombre=nombre)  # ✅ Seguro
    return JsonResponse({"usuarios": list(usuarios.values())})
```

✔️ **Si necesitas usar SQL crudo, usa parámetros seguros:**  

```python
from django.db import connection

def buscar_usuario(request):
    nombre = request.GET.get("nombre")
    with connection.cursor() as cursor:
        cursor.execute("SELECT * FROM usuarios WHERE nombre = %s", [nombre])  # ✅ Seguro
        resultado = cursor.fetchall()
```

📌 **Evita concatenar strings en consultas SQL** y usa **placeholders (`%s`)** para prevenir inyecciones.  

---

## ✅ **Resumen de protecciones en Django**  

| Tipo de ataque | Prevención en Django |
|---------------|----------------|
| **CSRF** | Usa `{% csrf_token %}` en formularios y `JWT` en APIs. |
| **XSS** | Django escapa variables automáticamente en plantillas. |
| **SQL Injection** | Usa consultas con ORM (`.filter()`) o parámetros seguros (`%s`). |